In [1]:
import os, sys, boto, mimetypes, zipfile, gzip
from io import StringIO, BytesIO
from optparse import OptionParser

import Quandl
import requests
import json
from collections import Counter

##Using the metadata to extract all of the data file names, we can extract all of the datasets

In [3]:
def makelist(num_pages):
    list_of_pages = []
    for i in range(1,num_pages+1):
        list_of_pages.append(i)
    return list_of_pages

##Make a list depending on the dataset size information. In the links for the metadata, it'll say how many times the dataset needs to be updated. For example, for the trading economics data, the metadata page: https://www.quandl.com/data/SGE/metadata shows how many requests need to be made in the bottom of the page (25 in this case)

In [5]:
#Getting all of the data file names from the metadata files (there are a certain number of pages)
#Each page that we can request has a limit of 300 dataset information, and so there are 25 pages in 
#trading economics database

In [38]:
list_of_pages_all = makelist('#put number here')
list_of_all_datasheets = []

for i in list_of_pages_all:
    url = '#put url here (without the page number after the = sign)'+str(i)+'&auth_token=X4GASyhZ_ecozx8hQLXm'
    response = requests.get(url)
    data = json.loads(response.content)
    for i in range(0,len(data[u'docs'])):
        list_of_all_datasheets.append(str(data[u'docs'][i][u'code']))

In [6]:
#EXAMPLE for a certain database:
#for i in list_of_pages_all:
#    url = 'https://www.quandl.com/api/v2/datasets.json?query=*&source_code=SGE&per_page=300&page='+str(i)+'&auth_token=X4GASyhZ_ecozx8hQLXm'
#    response = requests.get(url)
#    data = json.loads(response.content)
#    for i in range(0,len(data[u'docs'])):
#        if str(data[u'docs'][i][u'frequency']) == 'daily':
#            list_of_all_datasheets.append(str(data[u'docs'][i][u'code']))

#If you want to specifically look at daily frequency data, do an if statement to check frequency before appending datasheet name

In [42]:
len(list_of_all_datasheets) #Only the datasheets that are updated daily

17790

#Grabbing data and GOOGLE CLOUD STORAGE STUFF:

In [13]:
gs_access_key_id = 'GOOGZJ7NUP7VFYAGYTRL'
gs_secret_access_key = 'srwX2b70+Wa1afRe9kR22qH4XulBkrwCYZMNklmI'

In [19]:
conn = boto.connect_gs(gs_access_key_id,gs_secret_access_key)

In [27]:
buckets = conn.get_all_buckets()
list_of_buckets = []
for bucket in buckets:
  list_of_buckets.append(bucket.name)
list_of_buckets[0]

u'pushing_quandl'

In [28]:
bucket_name = list_of_buckets[0]
bucket = conn.get_bucket(bucket_name)

In [61]:
COMPRESSIBLE = [ 'text/plain', 'text/csv', 'application/xml',
                'application/javascript', 'text/css' ]
os.environ['AWS_ACCESS_KEY_ID'] = 'Your AWS access key id goes here'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'Your AWS secret access key goes here'

In [105]:
data_code = #a string that represents the database name. EXAMPLE: 'OFDP/' is the code for OFDP Database code

for i in range(len(list_of_all_datasheets)):
    data = Quandl.get(data_code+str(list_of_all_datasheets[i]),authtoken="X4GASyhZ_ecozx8hQLXm")
    data.to_csv('DATASHEETS/code_'+str(list_of_all_datasheets[i])+'.csv')
    content = open('DATASHEETS/code_'+str(list_of_all_datasheets[i])+'.csv')
    name = 'code_'+str(list_of_all_datasheets[i])+'.csv'
    key = bucket.new_key(name)
    type, encoding = mimetypes.guess_type(name)
    type = type or 'application/octet-stream'
    headers = { 'Content-Type': type, 'x-amz-acl': 'public-read' }
    states = [type]
        
    if type in COMPRESSIBLE:
        headers['Content-Encoding'] = 'gzip'
        compressed = BytesIO()
        gz = gzip.GzipFile(filename=name, fileobj=compressed, mode='w')
        gz.writelines(content)
        gz.close()
        content.close
        content = BytesIO(compressed.getvalue())
        states.append('gzipped')
    states = ', '.join(states)
    print '- %s =&gt; %s (%s)' % (name, key.name, states)
    key.set_contents_from_file(content, headers)
    content.close();

- OFDP_FUTURE_CL1.csv =&gt; OFDP_FUTURE_CL1.csv (text/csv, gzipped)
- OFDP_FUTURE_C1.csv =&gt; OFDP_FUTURE_C1.csv (text/csv, gzipped)
- OFDP_FUTURE_GC1.csv =&gt; OFDP_FUTURE_GC1.csv (text/csv, gzipped)
- OFDP_FUTURE_B1.csv =&gt; OFDP_FUTURE_B1.csv (text/csv, gzipped)
- OFDP_FUTURE_NG1.csv =&gt; OFDP_FUTURE_NG1.csv (text/csv, gzipped)
- OFDP_GOLD_2.csv =&gt; OFDP_GOLD_2.csv (text/csv, gzipped)
- OFDP_FUTURE_W1.csv =&gt; OFDP_FUTURE_W1.csv (text/csv, gzipped)
- OFDP_FUTURE_SK2011.csv =&gt; OFDP_FUTURE_SK2011.csv (text/csv, gzipped)
- OFDP_FUTURE_SF1997.csv =&gt; OFDP_FUTURE_SF1997.csv (text/csv, gzipped)
- OFDP_FUTURE_ES1.csv =&gt; OFDP_FUTURE_ES1.csv (text/csv, gzipped)
- OFDP_FUTURE_S1.csv =&gt; OFDP_FUTURE_S1.csv (text/csv, gzipped)
- OFDP_ALUMINIUM_21.csv =&gt; OFDP_ALUMINIUM_21.csv (text/csv, gzipped)
- OFDP_FUTURE_SH1994.csv =&gt; OFDP_FUTURE_SH1994.csv (text/csv, gzipped)
- OFDP_FUTURE_SQ1990.csv =&gt; OFDP_FUTURE_SQ1990.csv (text/csv, gzipped)
- OFDP_FUTURE_CT1.csv =&gt; OFDP_FUT

URLError: <urlopen error [Errno -2] Name or service not known>